In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Lib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text  import Tokenizer

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import SimpleRNN, Embedding, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint ,CSVLogger,EarlyStopping,LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall, F1Score, AUC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

## Read data

In [ ]:
df=pd.read_csv('/kaggle/input/spam-emails/spam.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['Category'].value_counts()

## Preprocessing

In [ ]:
encoder=LabelEncoder()
df["Category_Label"]=encoder.fit_transform(df["Category"])

df.head()

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['Message'])
tokenizer

In [ ]:
sequences=tokenizer.texts_to_sequences(df['Message'])
sequences[3]
#[6, 247, 150, 23, 381, 2995, 6, 139, 153, 57, 150]
#     ^-> refer to index in dic of tokenizer

In [ ]:
max_seq_len=max(len(i)for i in sequences)
print(max_seq_len)

In [ ]:
print(f'max is ',max_seq_len)

In [ ]:
x=pad_sequences(sequences,maxlen=max_seq_len)
x[0]

In [ ]:
x.shape

In [ ]:
y=df['Category_Label']
y

In [ ]:
y.shape

In [ ]:
#tokenizer.word_index     
                         #{'i': 1,
                         # 'to': 2,
                         #'you': 3,
                         #'a': 4,
                         #'the': 5,
                         #'u': 6,
                         #'and': 7,
                         #'in': 8,

In [ ]:
len(tokenizer.word_index)  #n of uniqe word

In [ ]:
vacab_size=len(tokenizer.word_index)+1  #+1 aas we add 0 in embedding

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

## RNN Model

In [ ]:
rnn_model= Sequential([
    Embedding(vacab_size,100,input_length=max_seq_len),
    SimpleRNN(128,activation='tanh'),
    #SimpleRNN(128,activation='tanh',return_seqences=True),
    #SimpleRNN(128,activation='tanh',return_seqences=True),
    #SimpleRNN(128,activation='tanh',return_seqences=False),
    Dense(1,activation='sigmoid')
])


In [ ]:
rnn_model.compile(optimizer=Adam(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy',Recall()])

In [ ]:
earlyStopping=EarlyStopping(monitor='val_loss',patience=7,restore_best_weights=True)

In [ ]:
hist=rnn_model.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_test, y_test),callbacks=[earlyStopping])

In [ ]:
rnn_model.evaluate(x_train,y_train)

In [ ]:
rnn_model.evaluate(x_test,y_test)

## LSTM Model

In [ ]:
LSTM_model= Sequential([
    Embedding(vacab_size,100,input_length=max_seq_len),
    LSTM(128,activation='tanh'),
    #LSTM(128,activation='tanh',return_seqences=True),
    #LSTM(128,activation='tanh',return_seqences=True),
    #LSTM(128,activation='tanh',return_seqences=False),
    Dense(1,activation='sigmoid')
])


In [ ]:
LSTM_model.compile(optimizer=Adam(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy',Recall()])

In [ ]:
earlyStopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [ ]:
hist_LSTM=LSTM_model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test),callbacks=[earlyStopping])

In [ ]:
LSTM_model.evaluate(x_train,y_train)

In [ ]:
LSTM_model.evaluate(x_test,y_test)

In [ ]:
tr_loss=hist.history['loss']
vall_loss=hist.history['val_loss']
tr_acc=hist.history['accuracy']
val_acc=hist.history['val_accuracy']
epochs=[i+1 for i in range(len(tr_loss))]

plt.figure(figsize=(6,6))
plt.subplot(1,2,1)
plt.plot(epochs,tr_loss,color='green',label='tr_loss')
plt.plot(epochs,vall_loss,color='red',label='vall_loss')
plt.title('loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs,tr_acc,color='green',label='tr_acc')
plt.plot(epochs,val_acc,color='red',label='val_acc')
plt.title('Acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend

plt.tight_layout()
plt.show

In [ ]:
y_pred = LSTM_model.predict(x_test)

# Convert predictions to binary (0 or 1) based on a threshold (e.g., 0.5)
y_pred_classes = (y_pred > 0.5).astype(int)

# Compute confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classes).ravel()

print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Negatives (TN): {tn}")

In [ ]:
y_pred